<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-07 12:43:50
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 15.17 K (0.11%)
Current PnL: -13.28 L (-9.25%)
CY Booked + Current PnL: -6.32 L (-4.4%)
-------------------
Total profit:  4.65 L
Total loss:  -17.93 L
-------------------
Total Booked + Current PnL: 20.66 L (17.73%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.73 L (52.42%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.41,H-LC,92.71,113850.0,10284.0,10429.0,0.48,9.93,9.16,20.00,32.0,0.99,0.83,25.40,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,87.50,110720.0,6408.0,11127.0,-1.07,6.14,10.05,16.81,27.0,0.58,0.81,23.18,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,-1.39,M-LC,30.21,173063.0,32223.0,12270.0,-0.57,22.88,7.09,31.59,88.0,2.63,1.27,50.33,XY24,NTT,BANKS
71,SBIN,760.30,863.00,-11.51,M-LC,31.25,214186.0,12706.0,14500.0,-0.44,6.31,6.77,13.51,77.0,0.88,1.57,19.70,XY25,NTT,BANKS
92,VIPIND,488.80,489.00,-211.74,H-SC,97.92,80603.0,-14713.0,14750.0,-1.23,-15.44,18.30,0.04,157.0,-1.00,0.59,58.98,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,35.94,151064.0,151.0,60305.0,-0.66,0.1,39.92,40.06,103.0,0.00,1.10,23.56,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,48.96,99900.0,-604.0,31239.0,-1.87,-0.6,31.27,30.49,97.0,-0.02,0.73,31.61,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-17.08,L-SC,100.00,166757.0,24915.0,31650.0,16.20,17.57,18.98,39.88,262.0,0.79,1.22,68.99,XY24,NTT,POWER
65,RELAXO,902.64,1176.00,-28.63,H-SC,95.83,64550.0,-50988.0,85974.0,8.51,-44.13,133.19,30.28,131.0,-0.59,0.47,26.11,X40N,NTT,FOOTWEAR
27,DEN,52.79,75.00,125.39,L-SC,89.58,63360.0,-21104.0,56638.0,4.79,-24.99,89.39,42.07,261.0,-0.37,0.46,34.56,AR,NTT,ENTERTAINMENT
26,DABUR,505.20,735.00,-7.00,X-MC,86.46,201240.0,4212.0,85406.0,4.22,2.14,42.44,45.49,56.0,0.05,1.47,14.31,XY24,BTT,FMCG
3,ALKYLAMINE,2347.98,4546.37,10.12,X-SC,98.96,102473.0,1510.0,93025.0,3.83,1.50,90.78,93.63,67.0,0.02,0.75,52.63,SR,ATH,CHEMICALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SATIN,199.77,274.00,-17.21,H-SC,69.79,164013.0,-31961.0,104788.0,-3.17,-16.31,63.89,37.16,135.0,-0.31,1.20,22.48,XY24,NTT,FINANCE
87,UJJIVANSFB,52.77,53.00,47.02,M-SC,26.04,127170.0,-15309.0,15934.0,-3.01,-10.74,12.53,0.44,247.0,-0.96,0.93,50.10,OX40N,NTT,BANKS
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,72.92,65568.0,-47981.0,88032.0,-2.82,-42.26,134.26,35.27,265.0,-0.55,0.48,60.48,XR,NTT,HOTELS
15,BLUESTARCO,1646.70,2337.55,-3.89,H-SC,96.88,180300.0,15630.0,53459.0,-2.07,9.49,29.65,41.95,124.0,0.29,1.32,17.67,X40N,ATH,AC
78,SURYODAY,189.78,240.00,53.51,H-SC,10.42,109257.0,-40669.0,80348.0,-2.03,-27.13,73.54,26.46,147.0,-0.51,0.80,39.97,XR,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,26.04,127170.0,-15309.0,15934.0,-3.01,-10.74,12.53,0.44,247.0,-0.96,0.93,50.10,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,29.17,150807.0,-20334.0,20284.0,1.81,-11.88,13.45,-0.03,191.0,-1.00,1.10,8.39,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,148949.0,-25610.0,25649.0,0.14,-14.67,17.22,0.02,133.0,-1.00,1.09,29.16,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.74,H-SC,97.92,80603.0,-14713.0,14750.0,-1.23,-15.44,18.30,0.04,157.0,-1.00,0.59,58.98,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,8.33,86450.0,-16202.0,16201.0,-0.55,-15.78,18.74,-0.00,59.0,-1.00,0.63,31.41,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.06,X-LC,44.79,266768.0,-29166.0,123087.0,0.01,-9.86,46.14,31.73,1.0,-0.24,1.95,6.78,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.47,X-LC,75.00,293685.0,32084.0,113539.0,1.36,12.26,38.66,55.67,2.0,0.28,2.15,19.12,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,61.46,269920.0,-427.0,35063.0,3.02,-0.16,12.99,12.81,5.0,-0.01,1.97,12.89,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,16.86,X-LC,65.62,267950.0,38717.0,28563.0,0.93,16.89,10.66,29.35,10.0,1.36,1.96,27.32,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-3.72,X-LC,21.88,81362.0,-28810.0,117885.0,-1.15,-26.15,144.89,80.85,11.0,-0.24,0.59,19.83,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2978.0,NaN,NaN,91.67,74450.0,40001.0,0.0,2.90,116.12,0.00,116.12,274.0,inf,0.54,13.85,AR,ATH,ELECTRICAL
88,UNIONBANK,123.87,163.0,-1.39,M-LC,30.21,173063.0,32223.0,12270.0,-0.57,22.88,7.09,31.59,88.0,2.63,1.27,50.33,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,40.62,229439.0,4477.0,163177.0,-1.10,1.99,71.12,74.53,189.0,0.03,1.68,33.21,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,26.30,H-SC,42.71,129238.0,3941.0,55133.0,-0.87,3.15,42.66,47.15,143.0,0.07,0.95,61.69,MH,ATH,POWER
20,CAMS,3643.00,5250.99,3.87,H-SC,28.12,116654.0,14650.0,30377.0,-1.55,14.36,26.04,44.14,126.0,0.48,0.85,32.94,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,30.21,173063.0,32223.0,12270.0,-0.57,22.88,7.09,31.59,88.0,2.63,1.27,50.33,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-3.31,M-LC,88.54,165974.0,15029.0,94423.0,-0.87,9.96,56.89,72.51,71.0,0.16,1.21,14.33,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2978.00,NaN,NaN,91.67,74450.0,40001.0,0.0,2.90,116.12,0.00,116.12,274.0,inf,0.54,13.85,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.43,M-SC,64.06,202860.0,31239.0,130824.0,0.17,18.20,64.49,94.43,201.0,0.24,1.48,49.72,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.31,M-LC,88.54,165974.0,15029.0,94423.0,-0.87,9.96,56.89,72.51,71.0,0.16,1.21,14.33,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.45,H-SC,67.71,134048.0,10712.0,123056.0,-0.69,8.69,91.80,108.45,123.0,0.09,0.98,32.98,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,26.30,H-SC,42.71,129238.0,3941.0,55133.0,-0.87,3.15,42.66,47.15,143.0,0.07,0.95,61.69,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,20.83,17783.0,-31810.0,734.0,1.79,-64.14,4.13,-62.66,270.0,-43.34,0.13,19.25,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,72.92,65568.0,-47981.0,88032.0,-2.82,-42.26,134.26,35.27,265.0,-0.55,0.48,60.48,XR,NTT,HOTELS
58,MASFIN,326.60,399.50,-18.70,H-SC,62.50,92250.0,-5730.0,27601.0,-0.89,-5.85,29.92,22.32,145.0,-0.21,0.67,33.79,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.80,X-LC,35.94,98010.0,265.0,37136.0,0.29,0.27,37.89,38.26,18.0,0.01,0.72,30.87,X40,BTT,FINANCE
3,ALKYLAMINE,2347.98,4546.37,10.12,X-SC,98.96,102473.0,1510.0,93025.0,3.83,1.50,90.78,93.63,67.0,0.02,0.75,52.63,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-17.08,L-SC,100.00,166757.0,24915.0,31650.0,16.20,17.57,18.98,39.88,262.0,0.79,1.22,68.99,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,10.12,X-SC,98.96,102473.0,1510.0,93025.0,3.83,1.50,90.78,93.63,67.0,0.02,0.75,52.63,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-211.74,H-SC,97.92,80603.0,-14713.0,14750.0,-1.23,-15.44,18.30,0.04,157.0,-1.00,0.59,58.98,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.89,H-SC,96.88,180300.0,15630.0,53459.0,-2.07,9.49,29.65,41.95,124.0,0.29,1.32,17.67,X40N,ATH,AC
65,RELAXO,902.64,1176.00,-28.63,H-SC,95.83,64550.0,-50988.0,85974.0,8.51,-44.13,133.19,30.28,131.0,-0.59,0.47,26.11,X40N,NTT,FOOTWEAR


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.94
1,25,41.92
2,50,70.58


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.44
LC    33.93
MC    21.58
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.89
X40      13.00
XY25     12.03
X40N     11.89
XR       10.04
OX40N     8.19
AR        8.16
MH        1.68
X5K       1.45
X200      1.41
SR        1.24
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.50
M-SC    15.89
H-LC    14.09
X-LC    12.92
H-MC     9.51
X-MC     7.65
M-LC     5.88
X-SC     4.42
M-MC     4.06
L-SC     2.63
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-0.56,34.20
IT,12.13,-9.63,63.50
FINANCE,8.51,-11.86,52.45
BANKS,7.77,-7.47,51.32
MISC,5.82,-20.88,77.13
PAINTS,5.72,-11.83,33.87
ELECTRICAL,5.46,1.56,44.42
HEALTHCARE,4.60,-2.51,32.45
AUTO,4.31,-14.49,64.83


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2668998.0,26
XR,1084641.0,13
AR,1025523.0,10
X40,677534.0,10
X40N,534471.0,11
XY25,473285.0,8
OX40N,309893.0,11
SR,174970.0,2
MH,86372.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1978293.0,20
M-SC,1770265.0,21
H-LC,612349.0,12
X-LC,541883.0,8
H-MC,533725.0,8
M-MC,415424.0,3
X-MC,404752.0,6
X-SC,315763.0,5
M-LC,257707.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      885137.0      6
M-SC       XY24      811595.0      7
H-SC       AR        525492.0      3
M-SC       XR        336413.0      4
M-MC       XY24      305354.0      2
H-MC       XY24      298583.0      3
X-LC       X40       273762.0      3
H-SC       XR        216645.0      3
           X40N      208569.0      4
H-LC       X40N      196508.0      4
M-SC       AR        182609.0      2
           OX40N     164949.0      5
H-LC       X40       155031.0      3
X-MC       X40       149689.0      2
L-SC       XR        149433.0      2
M-LC       XY24      148784.0      2
X-SC       XR        123826.0      1
M-SC       XY25      121665.0      1
X-LC       AR        117885.0      1
M-MC       XR        110070.0      1
X-LC       XY25      103732.0      3
X-MC       XY25      100471.0      1
M-LC       XR         94423.0      1
X-SC       SR         93025.0      1
H-SC       OX40N      87317.0      3
H-MC       XY25       85730.0      1
X-MC       XY24       85406.0      1
H-LC       AR         82594.0      1
M-SC       SR         81945.0      1
           X40        71089.0      1
H-LC       X5K        69395.0      2
           X200       67867.0      1
H-MC       AR         60305.0      1
X-SC       XY24       57486.0      1
L-SC       AR         56638.0      1
H-SC       MH         55133.0      1
L-MC       XR         53831.0      1
X-MC       X40N       52985.0      1
L-LC       XY25       47187.0      1
X-LC       X40N       46504.0      1
X-SC       OX40N      41426.0      2
H-LC       XY24       40954.0      1
L-SC       XY24       31650.0      1
H-MC       MH         31239.0      1
           X40N       29905.0      1
           X40        27963.0      1
X-MC       OX40N      16201.0      1
M-LC       XY25       14500.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
